In [116]:
import json
import numpy as np
import itertools


# trainset
with open("train.json", encoding='utf-8') as f:
    train_data = json.load(f)
print("num_dishes:" + str(len(train_data)))
cusines = []
for d in data:
    cusines.append(d['cuisine'])
# print(np.unique(cusines))
print("num_unique_cuisine:" + str(len(np.unique(cusines))))
ingredients = []
for d in train_data:
    ingredients.append(d['ingredients'])
ingredients = list(itertools.chain.from_iterable(ingredients))
unique_ingredients = np.unique(ingredients)
print(len(ingredients))
print("num_unique_ingredients:" + str(len(unique_ingredients)))

# testset
test_ids = []
with open("test.json", encoding='utf-8') as f:
    test_data = json.load(f)
for d in test_data:
    test_ids.append(d['id'])
print("test_num_dishes:" + str(len(test_data)))

num_dishes:39774
num_unique_cuisine:20
428275
num_unique_ingredients:6714
test_num_dishes:9944


In [78]:
## Q2(c)
# make ingredients dict
ingred_dict = {}
ind = 0
for i in unique_ingredients:
    ingred_dict[i] = ind
    ind = ind + 1
# Feature matrix
F = np.zeros((len(data), len(unique_ingredients)), dtype=int)
for row in range(len(data)):
    for ing in data[row]["ingredients"]:
        F[row, ingred_dict[ing]] = 1
F_test = np.zeros((len(test_data), len(unique_ingredients)), dtype=int)
for row in range(len(test_data)):
    for ing in test_data[row]["ingredients"]:
        if ing in ingred_dict:
            F_test[row, ingred_dict[ing]] = 1

In [106]:
## Q2(d)
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.model_selection import cross_validate
X = F[:,:]
Y = cusines[:]
model = BernoulliNB()
cv_result = cross_validate(model,X,Y,cv=3,return_train_score=True)
print(cv_result)
print("mean train score:" + str(np.mean(cv_result['train_score'])))
print("mean test score:" + str(np.mean(cv_result['test_score'])))

{'fit_time': array([6.76077056, 6.20061898, 5.35556746]), 'score_time': array([2.20452738, 2.18972325, 2.06335425]), 'test_score': array([0.68302797, 0.68225356, 0.68548144]), 'train_score': array([0.71400551, 0.71484066, 0.71412412])}
mean train score:0.7143234289165253
mean test score:0.6835876576455521


In [101]:
print(F.shape[0])

39774


In [112]:
X = F[:,:]
Y = cusines[:]
model = GaussianNB()
cv_result = cross_validate(model,X,Y,cv=3,return_train_score=True)
print(cv_result)
print("mean train score:" + str(np.mean(cv_result['train_score'])))
print("mean test score:" + str(np.mean(cv_result['test_score'])))

{'fit_time': array([7.48079348, 8.34969711, 8.38073444]), 'score_time': array([41.60915065, 41.4692111 , 41.96931434]), 'test_score': array([0.37925055, 0.38185384, 0.377377  ]), 'train_score': array([0.56097469, 0.54697341, 0.55256014])}
mean train score:0.5535027465935979
mean test score:0.3794937938208349


In [111]:
from sklearn.linear_model import LogisticRegression
X = F[:,:]
Y = cusines[:]
model = LogisticRegression()
cv_result = cross_validate(model,X,Y,cv=3,return_train_score=True)
print(cv_result)
print("mean train score:" + str(np.mean(cv_result['train_score'])))
print("mean test score:" + str(np.mean(cv_result['test_score'])))

{'fit_time': array([ 9.53297019, 10.35571456, 10.7504704 ]), 'score_time': array([0.82211399, 0.85048652, 0.81384802]), 'test_score': array([0.774787  , 0.77366317, 0.77882584]), 'train_score': array([0.87880502, 0.87784273, 0.87719629])}
mean train score:0.877948014910657
mean test score:0.7757586704089684


In [115]:
## Q2(g)
X = F[:,:]
Y = cusines[:]
result = LogisticRegression().fit(X,Y).predict(F_test)
print(result)

['british' 'southern_us' 'italian' ... 'italian' 'southern_us' 'mexican']


In [120]:
import csv
plabels = np.reshape(result, [len(result),1])
ids = np.reshape(test_ids, [len(test_ids),1])
output = np.concatenate((ids, plabels), axis = 1)
with open("result.csv","w",newline='') as csvfile: 
    writer = csv.writer(csvfile)
    writer.writerow(["id","cuisine"])
    writer.writerows(output)